In [1]:
import urllib.request 
import os
import zipfile
import pandas as pd

In [2]:
## Download and unzip COVID cases
temporary_folder = os.path.join(os.getcwd(), 'tmp')
if not os.path.exists(temporary_folder):
    os.makedirs(temporary_folder)
    
remote_file_name = "https://github.com/CSSEGISandData/COVID-19/archive/master.zip"
base_name = os.path.basename(remote_file_name)
local_file_name = os.path.join(temporary_folder, base_name)

urllib.request.urlretrieve(remote_file_name, local_file_name)

data_folder = os.path.join(os.getcwd(), 'world_data') 
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
    
with zipfile.ZipFile(local_file_name, 'r') as zip_ref:
    zip_ref.extractall(data_folder)
    
#!unzip master.zip

In [3]:
## Download world population
remote_file_name = "http://api.worldbank.org/v2/en/indicator/SP.POP.TOTL?downloadformat=excel"
world_population_file = os.path.join(temporary_folder, "world_population.xls")

# The following statement is throwing an error but the download is being performed
urllib.request.urlretrieve(remote_file_name, world_population_file) 

('/Users/wcunha69/Documents/projects/COVID-19/tmp/world_population.xls',
 <http.client.HTTPMessage at 0x113dc9dd0>)

In [4]:
## Load world population DataFrame
world_population = pd.read_excel(world_population_file, sheet_name="Data", dtype=object, skiprows=3)
world_population = world_population[["Country Name", "Country Code", "2018"]]
world_population = world_population.rename(
    {"Country Name":"country","Country Code":"country_code", "2018":"population"}, 
    axis='columns') 

world_population['population'] = world_population['population'].fillna(0)
world_population['population'] = world_population['population'].astype('int')
print(world_population)


          country country_code  population
0           Aruba          ABW      105845
1     Afghanistan          AFG    37172386
2          Angola          AGO    30809762
3         Albania          ALB     2866376
4         Andorra          AND       77006
..            ...          ...         ...
259        Kosovo          XKX     1845300
260   Yemen, Rep.          YEM    28498687
261  South Africa          ZAF    57779622
262        Zambia          ZMB    17351822
263      Zimbabwe          ZWE    14439018

[264 rows x 3 columns]


In [5]:
## Load COVID cases 
covid_folder = os.path.join(data_folder, 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/')

li = []
for covid_file in os.listdir(covid_folder):
    if covid_file.find(".csv") > -1:
        df = pd.read_csv(os.path.join(covid_folder, covid_file), index_col=None, header=0)
        li.append(df)

covid_cases = pd.concat(li, axis=0, ignore_index=True)
covid_cases = covid_cases.rename(
    {
        "Province/State":"province_state",
        "Country/Region":"country", 
        "Last Update":"last_update",
        "Confirmed":"confirmed",
        "Deaths":"deaths",
        "Recovered":"recovered",
        "Latitude":"latitude",
        "Longitude":"longitude"
    }, axis='columns'
) 


print(covid_cases)
#print(recent)
#covid = pd.read_csv(covid)
#print(covid)

                            province_state         country  \
0                                    Hubei  Mainland China   
1                                Guangdong  Mainland China   
2                                    Henan  Mainland China   
3                                      NaN     South Korea   
4                                 Zhejiang  Mainland China   
...                                    ...             ...   
7009                    Northern Territory       Australia   
7010  Lackland, TX (From Diamond Princess)              US   
7011                 Montgomery County, TX              US   
7012     Omaha, NE (From Diamond Princess)              US   
7013    Travis, CA (From Diamond Princess)              US   

              last_update  confirmed  deaths  recovered  latitude  longitude  
0     2020-02-26T14:13:10    65187.0  2615.0    20969.0       NaN        NaN  
1     2020-02-26T10:33:02     1347.0     7.0      851.0       NaN        NaN  
2     2020-02-26T1

In [6]:
## Adjusting country names and merging dataframes
world_population = world_population.replace("Korea, Rep.", "South Korea")
world_population = world_population.replace("Hong Kong SAR, China", "Hong Kong")
world_population = world_population.replace("Iran, Islamic Rep.", "Iran")
world_population = world_population.replace("Russian Federation", "Russia")
world_population = world_population.replace("Macao SAR, China", "Macau")
world_population = world_population.replace("Venezuela, RB", "Venezuela")
world_population = world_population.replace("Bahamas, The", "The Bahamas")
world_population = world_population.replace("Egypt, Arab Rep.", "Egypt")
world_population = world_population.replace("Slovak Republic", "Slovakia")
covid_cases = covid_cases.replace("Mainland China", "China")
covid_cases = covid_cases.replace("US", "United States")
covid_cases = covid_cases.replace("UK", "United Kingdom")
covid_cases = covid_cases.replace("Iran (Islamic Republic of)", "Iran")
covid_cases = covid_cases.replace("Russian Federation", "Russia")
covid_cases = covid_cases.replace("Republic of Korea", "South Korea")
covid_cases = covid_cases.replace("Hong Kong SAR", "Hong Kong")
covid_cases = covid_cases.replace("Viet Nam", "Vietnam")
covid_cases = covid_cases.replace("Macao SAR", "Macau")
covid_cases = covid_cases.replace("Korea, South", "South Korea")
covid_cases = covid_cases.replace("North Ireland", "United Kingdom")
covid_cases = covid_cases.replace("Republic of Ireland", "Ireland")

covid_cases_pop = pd.merge(covid_cases, world_population, how='left', on='country')
print(((covid_cases_pop[covid_cases_pop['population'].isnull()])["country"]).drop_duplicates())

9                                 Others
39                                Taiwan
421                  Taipei and environs
436       occupied Palestinian territory
490                        French Guiana
506                  Republic of Moldova
516                           Martinique
518                         Saint Martin
527                               Brunei
531                             Holy See
541                     Saint Barthelemy
577                          Cruise Ship
616                              Czechia
634                              Taiwan*
747                     Congo (Kinshasa)
757                              Reunion
992                            Palestine
1144                        Vatican City
2347                          Guadeloupe
2414                         Saint Lucia
2423                 Congo (Brazzaville)
2433                             Mayotte
2435    Saint Vincent and the Grenadines
2441                          The Gambia
2446            

In [33]:
## Remove countries with no population 
covid_cases_pop['population'] = covid_cases_pop['population'].fillna(0)
covid_cases_pop = covid_cases_pop.loc[covid_cases_pop['population'] != 0]
covid_cases_pop['population'] = covid_cases_pop['population'].astype('int')

covid_cases_aggr = covid_cases_pop[["country", "last_update", "population", "confirmed", "deaths"]]
covid_cases_aggr["last_update"] = pd.to_datetime(covid_cases_aggr["last_update"])
covid_cases_aggr["last_update"] = covid_cases_aggr["last_update"].dt.date




/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
## Aggregate by country, date and population
covid_cases_aggr = covid_cases_aggr.groupby(["country", "last_update", "population"]).sum()
covid_cases_aggr = covid_cases_aggr.reset_index()

In [43]:
## Add rate columns
covid_cases_aggr["cases_per_million"] = covid_cases_aggr["confirmed"] / (covid_cases_aggr["population"] / 1000000)
covid_cases_aggr["fatalities_per_million"] = covid_cases_aggr["deaths"] / (covid_cases_aggr["population"] / 1000000)
covid_cases_aggr["fatalities_per_cases"] = covid_cases_aggr["deaths"] / covid_cases_aggr["confirmed"]

    country last_update  population  confirmed  deaths  cases_per_million  \
216  Brazil  2020-01-23   209469333        0.0     0.0           0.000000   
217  Brazil  2020-02-26   209469333        3.0     0.0           0.014322   
218  Brazil  2020-02-29   209469333        8.0     0.0           0.038192   
219  Brazil  2020-03-04   209469333        8.0     0.0           0.038192   
220  Brazil  2020-03-06   209469333       26.0     0.0           0.124123   
221  Brazil  2020-03-08   209469333       20.0     0.0           0.095479   
222  Brazil  2020-03-09   209469333       25.0     0.0           0.119349   
223  Brazil  2020-03-10   209469333       31.0     0.0           0.147993   
224  Brazil  2020-03-11   209469333      189.0     0.0           0.902280   
225  Brazil  2020-03-12   209469333       52.0     0.0           0.248246   
226  Brazil  2020-03-13   209469333      151.0     0.0           0.720869   
227  Brazil  2020-03-15   209469333      162.0     0.0           0.773383   

In [62]:

countries = ((covid_cases_aggr[covid_cases_aggr["country"] == "Brazil"])["country"]).drop_duplicates()
li = []

for country in countries:
    country_cases = covid_cases_aggr[covid_cases_aggr["country"] == country]
    country_cases = country_cases[country_cases["confirmed"] > 0]
    country_dates = pd.date_range(start=country_cases.last_update.min(), end=covid_cases_aggr.last_update.max())
    country_cases = country_cases.set_index('last_update').reindex(country_dates).fillna(method='pad').rename_axis('last_update').reset_index()
    li.append(country_cases)

covid_cases_final = pd.concat(li, axis=0, ignore_index=True)
print(covid_cases_final)


#print(covid_cases_aggr["population"])

   last_update country   population  confirmed  deaths  cases_per_million  \
0   2020-02-26  Brazil  209469333.0        3.0     0.0           0.014322   
1   2020-02-27  Brazil  209469333.0        3.0     0.0           0.014322   
2   2020-02-28  Brazil  209469333.0        3.0     0.0           0.014322   
3   2020-02-29  Brazil  209469333.0        8.0     0.0           0.038192   
4   2020-03-01  Brazil  209469333.0        8.0     0.0           0.038192   
5   2020-03-02  Brazil  209469333.0        8.0     0.0           0.038192   
6   2020-03-03  Brazil  209469333.0        8.0     0.0           0.038192   
7   2020-03-04  Brazil  209469333.0        8.0     0.0           0.038192   
8   2020-03-05  Brazil  209469333.0        8.0     0.0           0.038192   
9   2020-03-06  Brazil  209469333.0       26.0     0.0           0.124123   
10  2020-03-07  Brazil  209469333.0       26.0     0.0           0.124123   
11  2020-03-08  Brazil  209469333.0       20.0     0.0           0.095479   